## 새로운 데이터 전처리 후 json으로 만들기
- https://github.com/boostcampaitech2/data-annotation-cv-level3-cv-07/blob/master/utils/annotation-polygon.py
- https://github.com/boostcampaitech2/data-annotation-cv-level3-cv-07/blob/master/utils/annotation_for_revised.py

In [9]:
import os.path as osp
import json
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import copy

In [62]:
new_data_dir = "/opt/ml/input/data/AIStages_ANN"

In [76]:
with open(osp.join(new_data_dir, 'dataset/annotation.json'), 'r') as f:
    new_data = json.load(f)

In [77]:
new_data = new_data["images"]
len(new_data)

1288

In [78]:
new_data_tmp = copy.deepcopy(new_data)

In [79]:
cnt = 0
cnt_normal = 0
cnt_abnormal = 0

In [80]:
for img_name, img_info in tqdm(new_data.items()):
    # 라벨링이 되어 있지 않은 경우 -> abnormal
    # if img_info["words"] == {}:
    #     del new_data_tmp[img_name]
    #     cnt_abnormal += 1
    #     continue
    
    # 이미지에 단어가 10개 미만인 경우 -> abnormal 
    if len(img_info["words"]) < 10: 
        del new_data_tmp[img_name]
        cnt_abnormal += 1
        continue
    
    for word, word_info in img_info["words"].items():
        # transcription이 없는 경우 or 공백이 있는 경우 제거 -> abnormal
        if word_info['illegibility'] == False:
            if word_info['transcription'] == "" or len(word_info['transcription'].split(' ')) > 1: 
                del new_data_tmp[img_name]["words"][word]
                # word를 삭제하여 해당 image의 word가 모두 없어졌다면 image 자체를 삭제
                if new_data_tmp[img_name]["words"] == {}:
                    del new_data_tmp[img_name]
                    cnt_abnormal += 1
                continue
        
        # polygon을 이루는 point가 4개인 경우 -> normal
        if len(img_info["words"][word]["points"]) == 4:
            cnt_normal += 1
            continue
        
        # polygon을 이루는 point가 4개 미만인 경우 -> abnormal
        elif len(img_info["words"][word]["points"]) < 4:
            del new_data_tmp[img_name]["words"][word]
            
            # word를 삭제하여 해당 image의 word가 모두 없어졌다면 image 자체를 삭제
            if new_data_tmp[img_name]["words"] == {}:
                del new_data_tmp[img_name]
                cnt_abnormal += 1
                continue
                
            
        # polygon을 이루는 point가 4개 초과인 경우 -> rect 단위로 잘라서 넣는다.
        else:
            polygon_tmp = copy.deepcopy(new_data_tmp[img_name]["words"][word])
            polygon_to_rect = copy.deepcopy(polygon_tmp)
            polygon_to_rect["points"] = []
            for idx in range(len(img_info["words"][word]["points"]) // 2 - 1):
                polygon_to_rect["points"].append(polygon_tmp["points"][idx])
                polygon_to_rect["points"].append(polygon_tmp["points"][idx+1])
                polygon_to_rect["points"].append(polygon_tmp["points"][-idx-1])
                polygon_to_rect["points"].append(polygon_tmp["points"][-idx])
                new_data_tmp[img_name]["words"][word+f"{idx+100}"] = copy.deepcopy(polygon_to_rect)
                polygon_to_rect["points"] = []
            del new_data_tmp[img_name]["words"][word]
        
            # word를 삭제하여 해당 image의 word가 모두 없어졌다면 image 자체를 삭제
            if new_data_tmp[img_name]["words"] == {}:
                del new_data_tmp[img_name]
                cnt_abnormal += 1
                continue
            
            cnt += 1


100%|██████████| 1288/1288 [00:00<00:00, 11664.83it/s]


In [81]:
print(f'Normal polygon count : {cnt_normal}')
print(f'Deleted {cnt} over polygon')
print(f'Abnormal polygon count : {cnt_abnormal}')

Normal polygon count : 19037
Deleted 760 over polygon
Abnormal polygon count : 618


In [59]:
# new_data_tmp

In [82]:
new_data = { "images": new_data_tmp }

In [83]:
len(new_data["images"])

670

In [84]:
ufo_dir = osp.join(new_data_dir, 'ufo')
if not osp.exists(ufo_dir):
        os.makedirs(ufo_dir)

In [85]:
with open(osp.join(ufo_dir, 'train.json'), "w") as f:
    json.dump(new_data, f, indent="\t")